In [4]:
import pandas as pd
import requests
import json

In [16]:
file_request = requests.get("https://openfecdata.blob.core.windows.net/fecfilings/processed_output/cross_contributions.json/part-00000")
file_lines = [json.loads(l) for l in file_request.text.split('\n') if l]

In [41]:
df = pd.DataFrame(file_lines)
df_column_map = {
    'COUNT(contributions)': "NumContributions",
    'SUM(contributions)': "SumContributions",
    'district': "SourceDistrict",
    'state': "SourceState"
}
df.columns = [df_column_map.get(c, c) for c in df.columns]
df.CAND_OFFICE_DISTRICT = df.CAND_OFFICE_DISTRICT.apply(int)
df.SourceDistrict = df.SourceDistrict.apply(int)
df["WithinDistrict"] = (df.CAND_OFFICE_DISTRICT == df.SourceDistrict) & (df.CAND_OFFICE_ST == df.SourceState)

In [44]:
df[(df.CAND_OFFICE_DISTRICT==11) & (df.CAND_OFFICE_ST == "NC")].sort_values("SumContributions", ascending=False).head(5)

,CAND_OFFICE_DISTRICT,CAND_OFFICE_ST,NumContributions,SumContributions,SourceDistrict,SourceState,WithinDistrict
15762,11,NC,22,25219.38,11,NC,True
27160,11,NC,6,17550.00,33,CA,False
27503,11,NC,7,14150.00,37,CA,False
20651,11,NC,3,10690.00,9,NJ,False
33084,11,NC,5,9960.00,10,NC,False


In [55]:
df[df.CAND_OFFICE_ST == "VA"].groupby(["CAND_OFFICE_ST", "CAND_OFFICE_DISTRICT", "WithinDistrict"]).agg({"NumContributions": sum, "SumContributions": sum})

SumContributions  \
CAND_OFFICE_ST CAND_OFFICE_DISTRICT WithinDistrict                     
VA             0                    False                   60350.00   
               1                    False                  266300.00   
                                    True                    61320.40   
               2                    False                   95247.77   
                                    True                   102813.05   
               3                    False                   79300.00   
                                    True                    15525.00   
               4                    False                  175225.00   
                                    True                    47950.00   
               5                    False                  203256.73   
                                    True                   142367.63   
               6                    False                  347218.69   
                                    True                    10723.43   
               7                    False                  277501.60   
                                    True                    47013.66   
               8                    False                  310532.28   
                                    True                   157863.62   
               9                    False                  136975.00   
                                    True                    35175.00   
               10                   False                 1430625.40   
                                    True                   616084.00   
               11                   False                  249126.09   
                                    True                    72030.94   

                                                    NumContributions  
CAND_OFFICE_ST CAND_OFFICE_DISTRICT WithinDistrict                    
VA             0                    False                         31  
               1                    False                         78  
                                    True                          30  
               2                    False                         42  
                                    True                          14  
               3                    False                         57  
                                    True                          18  
               4                    False                        126  
                                    True                          24  
               5                    False                        149  
                                    True                          51  
               6                    False                        150  
                                    True                          10  
               7                    False                        135  
                                    True                          36  
               8                    False                        101  
                                    True                          23  
               9                    False                         79  
                                    True                          22  
               10                   False                        291  
                                    True                          43  
               11                   False                        117  
                                    True                          32

In [60]:
df[df.CAND_OFFICE_ST == "NC"].groupby(["SourceState", "SourceDistrict", "WithinDistrict"]).agg({"NumContributions": sum, "SumContributions": sum})

SumContributions  NumContributions
SourceState SourceDistrict WithinDistrict                                    
AK          1              False                     250.00                 1
AL          1              False                    2000.00                 1
            2              False                    2000.00                 1
            5              False                     650.00                 1
            6              False                    8500.00                 4
            7              False                    8000.00                 3
AR          2              False                   10800.00                 1
AZ          1              False                    1100.00                 2
            2              False                    4000.00                 2
            3              False                    4000.00                 2
            6              False                    7400.00                 3
            7              False                    1174.20                 1
            9              False                    2274.20                 3
CA          2              False                    7700.00                 4
            3              False                     250.00                 1
            6              False                    4000.00                 3
            8              False                     400.00                 1
            11             False                    9000.00                 5
            12             False                   22500.00                 8
            14             False                   20300.00                 4
            15             False                    8750.00                 4
            17             False                    5000.00                 2
            18             False                   17800.00                 4
            24             False                    3000.00                 2
            28             False                    2250.00                 3
            30             False                    2750.00                 3
            31             False                     400.00                 1
            33             False                   42500.00                17
            34             False                   13122.00                 6
            35             False                     250.00                 1
...                                                     ...               ...
TX          18             False                    2800.00                 2
            20             False                    5000.00                 4
            21             False                    9650.00                 5
            23             False                    1000.00                 1
            24             False                   14250.00                10
            25             False                    5900.00                 3
            30             False                   22700.00                12
            31             False                     750.00                 2
            32             False                    8700.00                 9
            33             False                     750.00                 2
            35             False                    5000.00                 4
UT          3              False                    1000.00                 1
VA          1              False                    3250.00                 5
            2              False                     500.00                 1
            3              False                   12750.00                 7
            7              False                   12950.00                 7
            8              False                  358212.94               122
            10             False                   43800.00                35
            11             False                  104200.00      

In [62]:
df.pivot_table(index=["CAND_OFFICE_ST", "CAND_OFFICE_DISTRICT"], columns=["SourceState", "SourceDistrict"], values="SumContributions", aggfunc=sum, fill_value=0)

SourceState                                AK      AL                       \
SourceDistrict                              1       1          2         3   
CAND_OFFICE_ST CAND_OFFICE_DISTRICT                                          
AK             0                     77648.42       0       0.00      0.00   
AL             1                         0.00  217675   48550.00  17050.00   
               2                         0.00    4750  167561.88  37350.00   
               3                         0.00    1436   48100.00  69355.24   
               4                         0.00    1000   11500.00   2250.00   
               5                         0.00       0    3000.00      0.00   
               6                         0.00   42700   65100.00  28900.00   
               7                         0.00    2950    9200.00   1000.00   
AR             1                         0.00       0       0.00      0.00   
               2                         0.00       0    1000.00      0.00   
               3                         0.00       0       0.00      0.00   
               4                         0.00       0      14.33     18.51   
AZ             1                      1000.00       0       0.00      0.00   
               2                      2085.00     250    3239.00   5025.00   
               3                         0.00    2700    2700.00      0.00   
               4                         0.00       0       0.00   1500.00   
               5                         0.00       0       0.00      0.00   
               6                         0.00       0       0.00      0.00   
               7                         0.00    2000    2000.00      0.00   
               8                         0.00       0       0.00      0.00   
               9                         0.00    2000    2000.00      0.00   
CA             1                         0.00       0       0.00      0.00   
               2                         0.00       0       0.00      0.00   
               3                      3600.00       0       0.00      0.00   
               4                         0.00       0       0.00      0.00   
               5                         0.00       0       0.00      0.00   
               6                         0.00       0       0.00      0.00   
               7                         0.00       0       0.00      0.00   
               8                         0.00    1000    1000.00      0.00   
               9                         0.00       0       0.00      0.00   
...                                       ...     ...        ...       ...   
VA             6                         0.00       0       0.00      0.00   
               7                         0.00       0    2700.00   1000.00   
               8                         0.00       0       0.00      0.00   
               9                         0.00       0       0.00      0.00   
               10                        0.00       0    1000.00      0.00   
               11                        0.00       0       0.00      0.00   
VI             0                         0.00       0       0.00      0.00   
VT             0                         0.00       0       0.00      0.00   
WA             1                       250.00    1000    1000.00      0.00   
               2                       250.00       0       0.00      0.00   
               3                         0.00       0       0.00      0.00   
               4                         0.00       0       0.00      0.00   
               5                         0.00       0       0.00      0.00   
               6                      3677.00    2000    2000.00      0.00   
               7                         0.00       0       0.00      0.00   
               8                         0.00       0       0.00      0.00   
               9                         0.00       0       0.00    750.00   
               10     